In [4]:
from dataset import YouCookII
from dataset import collate_fn
from torch.utils.data import DataLoader
from loss import loss_RA_MIL

import torch

def train(model, num_actions, batch_size, epochs=25, lr=0.001, y=0.5):
    dataset = YouCookII(num_actions, "/h/sagar/ece496-capstone/datasets/ycii")
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    train_loss = np.zeros(epochs)
    train_accuracy = np.zeros(epochs)
    
    model.train()
    
    for epoch in range(epochs):
        for data in dataloader:
            _, bboxes_tensor, features_tensor, steps_list, _, entity_count_list, _, _ = data
            batch_size = len(data[0])
            
            # Zero out any gradients.
            optimizer.zero_grad()

            # Run inference (forward pass).
            loss_E, loss_V, loss_R, _, _, _, _, _ = model(batch_size, num_actions + 1, steps_list, features_tensor, bboxes_tensor, entity_count_list)

            # Loss from alignment.
            loss_ = loss_RA_MIL(y, loss_R, loss_E, loss_V)
            print(loss_)

            # Backpropagation (backward pass).
            loss_.backward()

            # Update parameters.
            optimizer.step()

            # TODO: save loss and accuracy at each epoch, plot (and checkpoint).
            train_loss[epoch] = loss_
        
    return train_loss

In [ ]:
from model import Model

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(device)
train_loss = train(model, 8, 2, epochs=10, lr=0.001)